In [3]:
class projeto_envio():
    
    def __init__(self):
        self.leitura_base()
        self.transform_data()
        self.documento()


    def leitura_base(self):    
        #Importação Biblioteca
        import pandas as pd
        
        #Path/Caminho do arquivo
        path_arquivo = '../database/Database_client.csv'
        
        #Leitura do arquivo .csv
        def leitura_arquivo(path, separador=';', idioma='latin-1'):
            
            arquivo = pd.read_csv(path, sep=separador,encoding=idioma)
            return arquivo
        #Arquivo "Lido"
        self.arquivo = leitura_arquivo(path=path_arquivo, idioma='latin-1')
         
    def transform_data(self):       
        #Tranformando a coluna "Telefone" no formato (00) 0000-0000,  Digamos que todos os DDD fossem iguais a 24, mas pode inserir outro valor na variavel DDD, que funcionara da mesma maneira
        def transformar_numero(dataframe,DDD='', coluna_a_transformar=''):
            dataframe[coluna_a_transformar] = f'({DDD})' + dataframe[coluna_a_transformar].astype(str)
            dataframe[coluna_a_transformar] = dataframe[coluna_a_transformar].apply(lambda x: f'{x[:9]}-{x[9:]}')
            return dataframe
        
        #Transformando a coluna "Data da Última Compra" em datetime
        def transformar_data(dataframe, formato='%d/%m/%Y', coluna_a_transformar=''):
            from pandas import to_datetime
            dataframe[coluna_a_transformar] = to_datetime(dataframe[coluna_a_transformar], format=formato)  
            return
            
        arquivo_transformado = self.arquivo.copy()
        transformar_numero(dataframe=arquivo_transformado, DDD=24, coluna_a_transformar='Telefone')
        transformar_data(dataframe=arquivo_transformado,coluna_a_transformar='Data da Última Compra')
        
        self.arquivo_transformado = arquivo_transformado.copy()
        
    def documento(self):
        import reportlab
        

In [4]:
x = projeto_envio()

,ID,Nome,Forma de Pagamento,E-mail,Telefone,Data da Última Compra,Endereço
0,1,João da Silva,Crédito,joao@email.com,(24)12345-6789,2023-05-10,"Rua das Flores, 123"
1,2,Maria Oliveira,Débito,maria@email.com,(24)98765-4321,2023-04-25,"Avenida Central, 456"
2,3,Carlos Santos,Dinheiro,carlos@email.com,(24)45678-9123,2023-03-15,"Praça da Liberdade, 789"
3,4,Ana Pereira,Pix,ana@email.com,(24)32165-4987,2023-02-28,"Travessa dos Sonhos, 56"
4,5,Sofia Lima,Transferência,sofia@email.com,(24)78932-1654,2023-01-10,"Rua das Estrelas, 789"
5,6,Diego Oliveira,Boleto,diego@email.com,(24)45612-3789,2022-12-05,"Avenida dos Ventos, 10"
6,7,Julia Pereira,Cartão Presente,julia@email.com,(24)98732-1654,2022-11-20,"Alameda das Árvores, 87"
7,8,Gabriel Santos,Cheque,gabriel@email.com,(24)78945-6123,2022-10-30,"Rua dos Rios, 654"
8,9,Laura Fernandes,PayPal,laura@email.com,(24)32178-9456,2022-09-15,"Avenida das Montanhas, 234"
9,10,Matheus Oliveira,Cartão de Crédito,matheus@email.com,(24)65478-9321,2022-08-28,"Praça dos Anjos, 3"
